In [10]:
import qutip as qt
import numpy as np
import lmfit as lm
import matplotlib.pyplot as plt
from lmfit import Model

π = np.pi
_ = %precision 3

In [23]:
pip install sympy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 1.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [108]:
from sympy import symbols
a, b, c, d, e, f, g, h, i, j, k, l, m, n, o, p, w1= symbols('a b c d e f g h i j k l m n o p w1')

# Create a 4x4 matrix using the symbolic variables
matrix = np.array([
    [a, b, c, d],
    [e, f, g, h],
    [i, j, k, l],
    [m, n, o, p]
])

In [82]:
matrix

array([[a, b, c, d],
       [e, f, g, h],
       [i, j, k, l],
       [m, n, o, p]], dtype=object)

In [83]:
a = qt.tensor(qt.destroy(2), qt.identity(2))
b = qt.tensor(qt.identity(2), qt.destroy(2))
adag = a.dag()
bdag = b.dag()

In [84]:
np.array(adag*a)

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [85]:
def task(one, two, three):
    m1 = np.dot(np.dot(one, two), three)
    m2 = m1.reshape((2, 2, 2, 2))
    m3 = np.trace(m2, axis1=1, axis2=3)
    return m3

In [86]:
task(adag, a, matrix)

array([[0, 0],
       [1.0*i + 1.0*n, 1.0*k + 1.0*p]], dtype=object)

In [87]:
task(matrix, adag, a)

array([[0, 1.0*c + 1.0*h],
       [0, 1.0*k + 1.0*p]], dtype=object)

In [88]:
task(bdag, b, matrix)

array([[1.0*f, 1.0*h],
       [1.0*n, 1.0*p]], dtype=object)

In [89]:
task(matrix, bdag, b)

array([[1.0*f, 1.0*h],
       [1.0*n, 1.0*p]], dtype=object)

In [90]:
task(adag, b, matrix)

array([[0, 0],
       [1.0*e, 1.0*g]], dtype=object)

In [91]:
task(matrix, adag, b)

array([[1.0*g, 0],
       [1.0*o, 0]], dtype=object)

In [92]:
task(bdag, a, matrix)

array([[1.0*j, 1.0*l],
       [0, 0]], dtype=object)

In [93]:
task(matrix, bdag, a)

array([[0, 1.0*b],
       [0, 1.0*j]], dtype=object)

In [118]:
#part 1
task(adag, a, matrix) - task(matrix, adag, a), task(bdag, b, matrix) - task(matrix, bdag, b)

(array([[-1.0*a*c - 1.0*a*h, 0],
        [1.0*a**2 + 1.0*a*f - 1.0*a*k - 1.0*a*p, 1.0*a*c + 1.0*a*h]],
       dtype=object),
 array([[0, 0],
        [0, 0]], dtype=object))

In [119]:
#part 2
task(adag, b, matrix) - task(matrix, adag, b) + task(bdag, a, matrix) - task(matrix, bdag, a)

array([[-1.0*b*c - 1.0*b*h, 0],
       [1.0*a*b + 1.0*b*f - 1.0*b*k - 1.0*b*p, 1.0*b*c + 1.0*b*h]],
      dtype=object)

In [120]:
#part 3
task(a, matrix, adag)-0.5*task(adag, a, matrix)-0.5*task(matrix, adag, a)

array([[0.5*a*c + 0.5*a*h, 0],
       [-0.5*a**2 - 0.5*a*f + 0.5*a*k + 0.5*a*p, -0.5*a*c - 0.5*a*h]],
      dtype=object)

In [115]:
p1 = task(adag, a, matrix) - task(matrix, adag, a)
p2 = task(adag, b, matrix) - task(matrix, adag, b) + task(bdag, a, matrix) - task(matrix, bdag, a)
p3 = task(a, matrix, adag)-0.5*task(adag, a, matrix)-0.5*task(matrix, adag, a)

In [122]:
p1

array([[-1.0*a*c - 1.0*a*h, 0],
       [1.0*a**2 + 1.0*a*f - 1.0*a*k - 1.0*a*p, 1.0*a*c + 1.0*a*h]],
      dtype=object)

In [121]:
tobloch(p1)

[2.0*a**2 + 2.0*a*f - 2.0*a*k - 2.0*a*p, 0, -2.0*a*c - 2.0*a*h - 1]

In [126]:
temp1 = matrix.reshape((2, 2, 2, 2))
matrix2 = np.trace(temp1, axis1=1, axis2=3)
matrix2

array([[a + f, c + h],
       [i + n, k + p]], dtype=object)

In [127]:
tobloch(matrix2)

[2*i + 2*n, 0, 2*a + 2*f - 1]

In [106]:
def tobloch(rho):
    a = rho[0,0]
    b = rho[1,0]
    c = rho[0,1]
    d = rho[1,1]
    
    x = 2*np.real(b)
    y = 2*np.imag(b)
    z = 2*a - 1
    
    return [x, y, z]

In [ ]:
tobloch()

In [48]:
np.array(adag*a)*matrix

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1.0*k, 0],
       [0, 0, 0, 1.0*p]], dtype=object)

In [53]:
matrix_A = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

# Reshape the matrix into a 2x2x2x2 tensor (assuming a tensor product structure)
tensor_A = matrix_A.reshape((2, 2, 2, 2))

# Perform the partial trace over the second subsystem
partial_trace_A = np.trace(tensor_A, axis1=1, axis2=3)

# Display the result
print(partial_trace_A)

[[ 7 11]
 [23 27]]
